In [2]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [3]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [4]:
modification_site = pd.read_csv("/home/yuan/Documents/PTM_N_link.csv")

In [7]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"pdb_site"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"pdb_site"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"pdb_site"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])

In [8]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()

In [9]:
unique_list = unique_list.reset_index(drop=True)

In [10]:
unique_list.loc[unique_list["PDB"]=="4x8l"]

,PDB,CHAIN


In [11]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]

In [12]:
PDB_ID = '3g8c'
PDB = PDBList()
retrieve_status = PDB.retrieve_pdb_file(pdb_code = PDB_ID, pdir = "/mnt/HD1/pdb_data", file_format="pdb")
p = PDBParser()
structure = p.get_structure("X","/mnt/HD1/pdb_data/"+"pdb"+PDB_ID+".ent")

Structure exists: '/mnt/HD1/pdb_data/pdb3g8c.ent' 


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7475.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7523.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7571.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7967.
  PDBConstructionWarning,


In [13]:
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
for index,oneRow in unique_list.iloc[573:,].iterrows():
    try:
        pdbID = oneRow["PDB"]
        chainOrder = oneRow["CHAIN"]
        retrieve_status = PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "/mnt/HD1/pdb_data", file_format="pdb")
        if os.path.exists(retrieve_status):
            p = PDBParser()
            structure = p.get_structure("X","/mnt/HD1/pdb_data/"+"pdb"+pdbID+".ent")
            if chainOrder in [x._id for x in list(structure.get_chains())]:
                oneChain = pd.DataFrame(columns = ["Seq","Residue","Center","Direction","CA_position"])#center here means center of R group
                if structure.header["resolution"] <= 3.0:
                    if chainOrder in [x.id for x in list(structure[0].get_chains())]:  # Chain information not in pdb file
                        for residue in structure[0][chainOrder]:
                            if residue.get_resname() in aminoAcidCodes:  # Only treat common amino acid
                                if residue.get_id()[2] == " ":  # ignore insertions
                                    if len(list(residue.get_atoms())) > 3:
                                        if residue.get_resname() != "GLY":  # Glysine as a special case
                                            point = vectors.Vector([0, 0, 0])
                                            for atom in residue:
                                                if (atom.get_name() not in backbone):
                                                    point = point + atom.get_vector()
                                            center = point.__div__(len(residue) - 4)
                                            cToRGroup = residue["CA"].get_vector() - center
                                            oneChain.loc[len(oneChain)] = [residue.get_id()[1], residue.get_resname(),center, cToRGroup,residue["CA"].get_vector()]
                                        else:
                                            center = residue["CA"].get_vector()
                                            cToRGroup = center - (residue["C"].get_vector() + residue["N"].get_vector() + residue["O"].get_vector()).__div__(3)
                                            oneChain.loc[len(oneChain)] = [residue.get_id()[1], residue.get_resname(),center, cToRGroup,residue["CA"].get_vector()]
                distanceMatrix = pd.DataFrame(columns=list(oneChain.iloc[:, 0]), index=list(oneChain.iloc[:,0]))
                angleMatrix = pd.DataFrame(columns=list(oneChain.iloc[:, 0]), index=list(oneChain.iloc[:, 0]))
                numResidue = len(oneChain)
                print(numResidue)
                for row in range(0, numResidue):
                    if row % 200 == 0:
                        print(str(row) + "th row")
                    for column in range(0, numResidue):#not symetric anymore
                        coordinatesSubstraction = list(oneChain.loc[row, "Center"] - oneChain.loc[column, "Center"])
                        pairDistance = sqrt(sum(list(map(lambda x: x * x, coordinatesSubstraction))))
                        distanceMatrix.iloc[row, column] = pairDistance
                        v1 = oneChain.loc[row, "Direction"]
                        v2 = oneChain.loc[column,"Center"] - oneChain.loc[row,"CA_position"] 
                        if abs(pairDistance-0)>0.00001: #Exclue 0 distance, e.g. self to self
                            pairAngle = 180 * np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))) / np.pi

                        else:
                            pairAngle = 0
                        angleMatrix.iloc[row, column] = pairAngle
                        if not np.isnan(pairDistance):
                            query = '''INSERT ignore INTO `Distance_angle_PTM` (`pdbID`, `chain`,`ID_1`,`Res_1`,`ID_2`,`Res_2`,`Distance`,`Angle`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s);'''
                            value = (pdbID,chainOrder,oneChain.iloc[row,0],oneChain.iloc[row,1],oneChain.iloc[column,0],oneChain.iloc[column,1],float(pairDistance),float(pairAngle))
                            conn.cursor().execute(query, value)
                    conn.commit()
    except Exception as e:
        print(e)
        print(pdbID, "skipped")

conn.close()

/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 16829.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 16899.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 16829.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 16899.
  PDBConstructionWarning,


0
Structure exists: '/mnt/HD1/pdb_data/pdb1s78.ent' 
0
506
0th row
200th row
400th row


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8642.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 8670.
  PDBConstructionWarning,


583
0th row
200th row
400th row


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10365.
  PDBConstructionWarning,


...

0
Structure exists: '/mnt/HD1/pdb_data/pdb3fxi.ent' 
0


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 24916.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 25056.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 25196.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 25360.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 25524.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/pyt

140
0th row
Structure exists: '/mnt/HD1/pdb_data/pdb4g8a.ent' 


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 24916.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 25056.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 25196.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 25360.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 25524.
  PDBConstructionWarning,
/home/yuan/anaconda3/envs/pdbPy3/lib/pyt

140
0th row


%%time
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
try:
    with conn as cursor: #auto commit; no close() called
        with cursor: # close() called here
            sql_select = "Select * FROM pdbdb.Distance_angle_PTM da "
            sql_where = "WHERE da.pdbID = \"3g8c\" and da.chain = \"A\" and da.ID_1=27 limit 1035;"    
            sql = sql_select+sql_where
            data = pd.read_sql_query(sql, conn)
            #print(data)
 
except Exception as e: # catch exceptions
    print("~~~~~~~~~~~~~~")
    print(e)
finally:
    if conn:
        conn.close()

In [6]:
def n_nearest_neighbour(PDB_ID,Chain,res_ID,n):
    
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain = \"{}\" and da.ID_1= \"{}\";".format(PDB_ID,Chain,res_ID)  
                sql = sql_select+sql_where
                data = pd.read_sql_query(sql, conn)
        return(data.groupby(["ID"]).first().sort_values("Distance").iloc[2:(n+2),]) #from 2 to exclude self vs self

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()

In [7]:
def n_nearest_neighbour_by_seq(PDB_ID,Chain,res_ID,n):
    if n >= res_ID:
        return []
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain = \"{}\" and da.ID_1= \"{}\";".format(PDB_ID,Chain,res_ID)  
                sql = sql_select+sql_where
                data = pd.read_sql_query(sql, conn)
        head_part = data.groupby(["ID"]).first().iloc[(res_ID-n-1):(res_ID-1),]
        tail_part = data.groupby(["ID"]).first().iloc[res_ID:(res_ID+n),]
        return(pd.concat([head_part,tail_part])) #from 2 to exclude self vs self

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()

In [8]:
def One_residue_retrieval(residue_1,PDB_ID,Chain):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain=\"{}\" and da.ID_1 = {};".format(PDB_ID,Chain,residue_1)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [9]:
def Get_FEATURE(PDBID,Chain,Seq):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`FEATURE` da "
                sql_where = "WHERE da.PDBID = \"{}\" and da.Chain=\"{}\" and da.Seq = {};".format(PDBID,Chain,Seq)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1[data_1["Component"]=="C"] # currently only get C component
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [10]:
def Get_PSSM(PDBID,Chain,Seq):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`PSSM_uniref50` da "
                sql_where = "WHERE da.PDBID = \"{}\" and da.Chain=\"{}\" and da.Seq = {};".format(PDBID,Chain,Seq)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [11]:
def Get_PSSM(PDBID,Chain,Seq):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`PSSM_uniref50` da "
                sql_where = "WHERE da.PDBID = \"{}\" and da.Chain=\"{}\" and da.Seq in {};".format(PDBID,Chain,Seq)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [12]:
def Get_Neigh_PSSM(PDBID,Chain,Seq,num_neigh):
    '''
    Retrieve redsidue‘s and neighbours' PSSM.
    No or imcomplete records will return an empty dataframe
    '''
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`PSSM_uniref50` da "
                sql_where = "WHERE da.PDBID = \"{}\" and da.Chain=\"{}\" and da.Seq = {};".format("XXX",Chain,Seq)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
        if Seq <= num_neigh:
            data = data_1
        else:
            with conn as cursor: #auto commit; no close() called
                with cursor: # close() called here
                    sql_select = "Select da.* FROM pdbdb.`PSSM_uniref50` da "
                    neigh = "(" + ",".join(map(str,list(range(Seq-num_neigh,Seq+num_neigh+1)))) + ")"
                    sql_where = "WHERE da.PDBID = \"{}\" and da.Chain=\"{}\" and da.Seq in {};".format(PDBID,Chain,neigh)    
                    sql = sql_select+sql_where
                    data_2 = pd.read_sql_query(sql, conn)
            if data_2.shape[0] < 2*num_neigh + 1:
                data = data_1
            else:
                data = data_2

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [137]:
pos_sample_and_neig = pd.DataFrame()
for key, value in modification_site_dic.items():
    for chain, sites in value.items():
        for site in sites:
            pos_sample_and_neig = pd.concat([pos_sample_and_neig,n_nearest_neighbour(key,chain,site,1)])

#### Now, positive sites are ready. Negative sites are going to be picked. 
Right now, one pos - one neg

In [16]:
neg_sample_and_neig = pd.DataFrame()
for i in range(len(pos_sample_and_neig)):
    result_step1 = One_residue_retrieval(pos_sample_and_neig.iloc[i,2],pos_sample_and_neig.iloc[i,0],pos_sample_and_neig.iloc[i,1])
    
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard

    result_step3 = result_step2[~result_step2.ID_2.isin(modification_site_dic[pos_sample_and_neig.iloc[i,0]][pos_sample_and_neig.iloc[i,1]])] # activate discard
    #print(result_step3)
    result_step4 = result_step3[result_step3["Res_2"]=="LYS"]
    #print(result_step4)
    #print(result_step4.sample(1,random_state = 1))
    neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(1)])

##### Res_2 in neg_sample_and_neig is the negative sample

In [18]:
pos_sample_and_neig_pruned = pos_sample_and_neig.iloc[:,[0,1,2,3]].copy(deep=True)
pos_sample_and_neig_pruned["type"] = "Pos"
pos_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [19]:
neg_sample_and_neig_pruned = neg_sample_and_neig.iloc[:,[0,1,4,5]].copy(deep=True)
neg_sample_and_neig_pruned["type"] = "Neg"
neg_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [20]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [21]:
neighbour_information_list = []
for i in range(0,len(pos_neg_sample)):
    neigh = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],5).iloc[:,5:8].values.flatten()
    neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))

In [13]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn.metrics import classification_report, roc_auc_score, roc_curve, make_scorer, confusion_matrix, \
    recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, GroupKFold, cross_val_score
from sklearn.metrics import auc, precision_recall_curve,accuracy_score
from sklearn.utils import shuffle
from sklearn import tree, svm, naive_bayes, neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder,LabelEncoder
from scipy import interp
import matplotlib.pyplot as plt

In [14]:
clfs = {'decision_tree': tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(),

        'K_neighbor': neighbors.KNeighborsClassifier(),
        'bagging_knn': BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5, max_features=0.5),
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5, max_features=0.5),
        'random_forest': RandomForestClassifier(n_estimators=200),
        'adaboost': AdaBoostClassifier(n_estimators=200),
        'gradient_boost': GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0),
        'svm': svm.SVC(C=0.9,kernel='rbf',gamma='auto')
        }
#'svm': svm.SVC(probability=True)

In [15]:


aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(aminoAcidCodes)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)


In [29]:
amino_acid_property_by_res = {"ALA":1,"ILE":1,"LEU":1,"MET":1,"VAL":1,
                      "PHE":2, "TRP":2,"TYR":2,
                       "ASN":3,"CYS":3,"GLN":3,"SER":3,"THR":3,
                      "ASP":4,"GLU":4,"ARG":5,"HIS":5,"LYS":5,"GLY":6,"PRO":6}
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform([1,2,3,4,5,6])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)


In [30]:
neighbour_information_list = []
for i in range(0,len(pos_neg_sample)):
    neigh = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],1).iloc[:,5:8].values.flatten()
    neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))

In [32]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    for element in each_res:
        if isinstance(element,str):
            one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
        else:
            one_res_data.append(element)
    train_dataset.append(one_res_data)
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<1):
        wrong_sample.append(i)

y_label =[1 for x in range(0,len(pos_sample_and_neig_pruned))]
y_label.extend([2 for i in range(0,len(neg_sample_and_neig_pruned))])

final_y_label = [y_label[i] for i in range(0,len(pos_neg_sample)) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,len(pos_neg_sample)) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

# skf = StratifiedKFold(n_splits=3)
# for train, test in skf.split(shuffle_train_dataset,shuffle_y_label):
#     for clf in clfs:
#         clfs[clf].fit(shuffle_train_dataset[train],shuffle_y_label[train])
#         conf_mat = confusion_matrix(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test])
#         print(clf)
#         print(accuracy_score(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test]))

for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,cv=10)
    print(sum(scores)/10)

svm


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/yuan/anaconda3/envs

0.591687449726
decision_tree
0.656243094237
naive_gaussian
0.509176694098
naive_mul
0.548862877268
K_neighbor
0.622952389852
bagging_knn
0.597847674802
bagging_tree
0.607180131353
random_forest
0.683932537874
adaboost
0.547591175018
gradient_boost
0.547948664686


# After using using CD-HIT to remove redundant samples

In [16]:
cd_hit_7 = pd.read_csv("retained_CDHIT_0.7_PTM.csv")

In [17]:
cd_hit_7.head()

,pdbID,Chain
0,1ln4,A
1,1lop,A
2,1m63,A
3,1m63,B
4,1m63,D


#### 5 neighbours 1:1 = pos:neg

In [18]:
pos_sample_and_neig = pd.DataFrame()
for key, value in modification_site_dic.items():
    for chain, sites in value.items():
        if chain in list(cd_hit_7[cd_hit_7["pdbID"]==key]["Chain"]):
            for site in sites:
                neigh = n_nearest_neighbour_by_seq(key,chain,site,1)
                if len(neigh) > 0:
                    pos_sample_and_neig = pos_sample_and_neig.append(neigh.iloc[0,:])

In [19]:
pos_sample_and_neig = pos_sample_and_neig[neigh.columns]
pos_sample_and_neig = pos_sample_and_neig.astype({"ID_1":int,"ID_2":int})

In [20]:
neg_sample_and_neig = pd.DataFrame()
for i in range(len(pos_sample_and_neig)):
    result_step1 = One_residue_retrieval(pos_sample_and_neig.iloc[i,2],pos_sample_and_neig.iloc[i,0],pos_sample_and_neig.iloc[i,1])
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard
    result_step3 = result_step2[~result_step2.ID_2.isin(modification_site_dic[pos_sample_and_neig.iloc[i,0]][pos_sample_and_neig.iloc[i,1]])] # activate discard
    result_step4 = result_step3[result_step3["Res_2"]=="LYS"]
    neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(1,replace = False)])

In [21]:
pos_sample_and_neig_pruned = pos_sample_and_neig.iloc[:,[0,1,2,3]].copy(deep=True)
pos_sample_and_neig_pruned["type"] = "Pos"
pos_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [22]:
neg_sample_and_neig_pruned = neg_sample_and_neig.iloc[:,[0,1,4,5]].copy(deep=True)
neg_sample_and_neig_pruned["type"] = "Neg"
neg_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [23]:
len(neg_sample_and_neig_pruned)

1036

In [24]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [71]:
neighbour_information_list = []
num_pos = 0
num_neg = 0
for i in range(0,len(pos_neg_sample)):
    features = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],\
                                       pos_neg_sample.iloc[i,2],60)
    if not isinstance(features,list):
        if pos_neg_sample.iloc[i,4] == "Pos":
            if features.shape[0] > 0:
                num_pos = num_pos + 1
                neighbour_information_list.append(np.insert(features.iloc[:,5:8].values.flatten(),0,pos_neg_sample.iloc[i,3]))
        else:
            if features.shape[0] > 0:
                num_neg = num_neg + 1
                neighbour_information_list.append(np.insert(features.iloc[:,5:8].values.flatten(),0,pos_neg_sample.iloc[i,3]))


In [72]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    i = 0
    for element in each_res[1:]:
        if isinstance(element,str):
#             one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
            pass
        else:
            one_res_data.append(element)
            
#         if isinstance(element,str):
#             one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
#         else:
#             if i % 2 == 1:
#                 one_res_data.append(element/50)
#             i = i + 1    
            

    train_dataset.append(one_res_data)
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<len(train_dataset[0])):
        wrong_sample.append(i)

y_label =[1 for x in range(0,num_pos)]
y_label.extend([2 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

for clf in clfs:
    print(clf)
    scores_acc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='accuracy',n_jobs=5,cv=10)
    scores_roc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='roc_auc',n_jobs=5,cv=10)
    scores_precision = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='precision',n_jobs=5,cv=10)
    scores_recall = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='recall',n_jobs=5,cv=10)
    print('accuracy:',"{:.4f}".format(sum(scores_acc)/10),'roc:',"{:.4f}".format(sum(scores_roc)/10), \
         'precision:',"{:.4f}".format(sum(scores_precision)/10),'recall:',"{:.4f}".format(sum(scores_recall)/10))

decision_tree
accuracy: 0.5554 roc: 0.5614 precision: 0.5531 recall: 0.5235
naive_gaussian
accuracy: 0.5170 roc: 0.5338 precision: 0.5127 recall: 0.6634
K_neighbor
accuracy: 0.5223 roc: 0.5287 precision: 0.5211 recall: 0.5524
bagging_knn
accuracy: 0.5326 roc: 0.5739 precision: 0.5315 recall: 0.5933
bagging_tree
accuracy: 0.5637 roc: 0.5862 precision: 0.5582 recall: 0.6674
random_forest
accuracy: 0.5968 roc: 0.6379 precision: 0.5796 recall: 0.6682
adaboost
accuracy: 0.5666 roc: 0.5766 precision: 0.5669 recall: 0.5632
gradient_boost
accuracy: 0.5437 roc: 0.5578 precision: 0.5454 recall: 0.5408
svm
accuracy: 0.6051 roc: 0.6051 precision: 0.5591 recall: 1.0000


In [73]:
pd.DataFrame(final_train_dataset).to_csv("/mnt/HD1/download/PTM_60_data.txt")
pd.DataFrame(final_y_label).to_csv("/mnt/HD1/download/PTM_60_label.txt")

### FEATURE training & testing

In [49]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [50]:
pos_neg_sample.head()

,pdbID,chain,ID,Res,type
740417,3jzf,A,27,LYS,Pos
777277,3jzf,A,124,LYS,Pos
936647,1o8c,A,50,LYS,Pos
988322,1o8c,A,209,LYS,Pos
1238230,2eck,A,13,LYS,Pos


In [53]:
neighbour_information_list = []
num_pos = 0
num_neg = 0
for i in range(0,len(pos_neg_sample)):
    features = Get_FEATURE(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],\
                pos_neg_sample.iloc[i,2])
    if pos_neg_sample.iloc[i,4] == "Pos":
        if features.shape[0] > 0:
            num_pos = num_pos + 1
            neighbour_information_list.append([float(x) for x in features["feature"].values[0].split(",")])
    else:
        if features.shape[0] > 0:
            num_neg = num_neg + 1
            neighbour_information_list.append([float(x) for x in features["feature"].values[0].split(",")])
        
#     print( n_nearest_neighbour_by_seq(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],2))
#     neigh = n_nearest_neighbour_by_seq(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],2).iloc[:,5:8].values.flatten()
#     neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))

/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1292, "Truncated incorrect DOUBLE value: '102A'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1292, "Truncated incorrect DOUBLE value: '105A'")
  result = self._query(query)
/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1292, "Truncated incorrect DOUBLE value: '115A'")
  result = self._query(query)


In [52]:
train_dataset = neighbour_information_list
# for each_res in neighbour_information_list:
#     one_res_data = []
#     for element in each_res[1:]:
#         if isinstance(element,str):
#             pass
#             #one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
#         else:
#             one_res_data.append(element)
#     train_dataset.append(one_res_data)
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<470):
        wrong_sample.append(i)

y_label =[1 for x in range(0,num_pos)]
y_label.extend([2 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

for clf in clfs:
    print(clf)
    scores_acc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='accuracy',n_jobs=5,cv=10)
    scores_roc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='roc_auc',n_jobs=5,cv=10)
    scores_precision = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='precision',n_jobs=5,cv=10)
    scores_recall = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='recall',n_jobs=5,cv=10)
    print('accuracy:',"{:.4f}".format(sum(scores_acc)/10),'roc:',"{:.4f}".format(sum(scores_roc)/10), \
         'precision:',"{:.4f}".format(sum(scores_precision)/10),'recall:',"{:.4f}".format(sum(scores_recall)/10))

decision_tree
accuracy: 0.5570 roc: 0.5571 precision: 0.5621 recall: 0.5069
naive_gaussian
accuracy: 0.5097 roc: 0.5392 precision: 0.6243 recall: 0.0579
K_neighbor
accuracy: 0.5410 roc: 0.5549 precision: 0.5414 recall: 0.5445
bagging_knn
accuracy: 0.5338 roc: 0.5538 precision: 0.5431 recall: 0.5617
bagging_tree
accuracy: 0.5811 roc: 0.5733 precision: 0.5669 recall: 0.6768
random_forest
accuracy: 0.6043 roc: 0.6538 precision: 0.5922 recall: 0.6584
adaboost
accuracy: 0.5420 roc: 0.5572 precision: 0.5416 recall: 0.5396
gradient_boost
accuracy: 0.5184 roc: 0.5431 precision: 0.5183 recall: 0.5233
svm
accuracy: 0.6115 roc: 0.6116 precision: 0.5630 recall: 1.0000


### PSSM training & testing

In [87]:
neighbour_information_list = []
num_pos = 0
num_neg = 0
for i in range(0,len(pos_neg_sample)):
    features = Get_Neigh_PSSM(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],\
                pos_neg_sample.iloc[i,2],6)
    if pos_neg_sample.iloc[i,4] == "Pos":
        if features.shape[0] > 0:
            num_pos = num_pos + 1
            neighbour_information_list.append(list(features.iloc[:,5:25].values.flatten()))
    else:
        if features.shape[0] > 0:
            num_neg = num_neg + 1
            neighbour_information_list.append(list(features.iloc[:,5:25].values.flatten()))
        


In [53]:
num_neg

938

In [54]:
train_dataset = neighbour_information_list
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<15):
        wrong_sample.append(i)

y_label =[1 for x in range(0,num_pos)]
y_label.extend([2 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

for clf in clfs:
    print(clf)
    scores_acc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='accuracy',n_jobs=5,cv=10)
    scores_roc = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='roc_auc',n_jobs=5,cv=10)
    scores_precision = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='precision',n_jobs=5,cv=10)
    scores_recall = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='recall',n_jobs=5,cv=10)
    print('accuracy:',"{:.4f}".format(sum(scores_acc)/10),'roc:',"{:.4f}".format(sum(scores_roc)/10), \
         'precision:',"{:.4f}".format(sum(scores_precision)/10),'recall:',"{:.4f}".format(sum(scores_recall)/10))

decision_tree
accuracy: 0.5556 roc: 0.5524 precision: 0.5691 recall: 0.5450
naive_gaussian
accuracy: 0.5082 roc: 0.5228 precision: 0.5131 recall: 0.3444
K_neighbor
accuracy: 0.5364 roc: 0.5464 precision: 0.5370 recall: 0.5407
bagging_knn
accuracy: 0.5316 roc: 0.5606 precision: 0.5386 recall: 0.5228
bagging_tree
accuracy: 0.5630 roc: 0.5743 precision: 0.5698 recall: 0.6575
random_forest
accuracy: 0.5843 roc: 0.6150 precision: 0.5659 recall: 0.6690
adaboost
accuracy: 0.5263 roc: 0.5396 precision: 0.5284 recall: 0.5186
gradient_boost
accuracy: 0.5141 roc: 0.5355 precision: 0.5161 recall: 0.5185
svm
accuracy: 0.6082 roc: 0.6379 precision: 0.5698 recall: 0.8950


## Deep Learning in Keras

In [91]:
train_dataset = neighbour_information_list
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<15):
        wrong_sample.append(i)

y_label =[0 for x in range(0,num_pos)]
y_label.extend([1 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

In [98]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    i = 0
    for element in each_res[1:]:
        if isinstance(element,str):
            one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
        else:
            one_res_data.append(element)
            
#         if isinstance(element,str):
#             pass
#         else:
#             if i % 2 == 1:
#                 one_res_data.append(element)
#             i = i + 1    
            

    train_dataset.append(one_res_data)
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<len(train_dataset[0])):
        wrong_sample.append(i)

y_label =[0 for x in range(0,num_pos)]
y_label.extend([1 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_neg+num_pos) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

In [69]:
from keras.models import Sequential
from keras.layers import Dense

In [92]:
# define the keras model
model = Sequential()
model.add(Dense(300, input_dim=260, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit the keras model on the dataset
model.fit(shuffle_train_dataset, shuffle_y_label, epochs=50, batch_size=20)


In [72]:
# evaluate the keras model
_, accuracy = model.evaluate(shuffle_train_dataset, shuffle_y_label)
print('Accuracy: %.2f' % (accuracy*100))

1881/1881 [==============================] - 0s 54us/step
Accuracy: 99.68


In [ ]:
# make class predictions with the model
predictions = model.predict_classes(shuffle_train_dataset)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (shuffle_train_dataset[i].tolist(), predictions[i], shuffle_y_label[i]))

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 590385714393151722
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3946922988132687430
physical_device_desc: "device: XLA_CPU device"
]


In [81]:
skf = StratifiedKFold(n_splits=10,shuffle=True)
skf.get_n_splits(shuffle_train_dataset,shuffle_y_label)

10

In [102]:
for train_index, test_index in skf.split(shuffle_train_dataset,shuffle_y_label):
    model = Sequential()
    model.add(Dense(100, input_dim=48, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(shuffle_train_dataset[train_index], \
              shuffle_y_label[train_index], epochs=50, batch_size=20,verbose=0)
    _, accuracy = model.evaluate(shuffle_train_dataset[test_index], shuffle_y_label[test_index]) 
    print('Accuracy: %.2f' % (accuracy*100))

208/208 [==============================] - 0s 290us/step
Accuracy: 56.73
208/208 [==============================] - 0s 272us/step
Accuracy: 52.88
207/207 [==============================] - 0s 257us/step
Accuracy: 55.07
207/207 [==============================] - 0s 296us/step
Accuracy: 52.17
207/207 [==============================] - 0s 283us/step
Accuracy: 57.00
207/207 [==============================] - 0s 270us/step
Accuracy: 51.69
207/207 [==============================] - 0s 276us/step
Accuracy: 55.07
207/207 [==============================] - 0s 287us/step
Accuracy: 57.97
207/207 [==============================] - 0s 274us/step
Accuracy: 58.45
207/207 [==============================] - 0s 293us/step
Accuracy: 57.00
